# Interactive Workflow on the Impact of Correlation and Standardization on Multidimensional Scaling

### The University of Texas at Austin, PGE 2020 SURI, Undergraduation Research
### Alan Scherman, Undergraduate Student, Rice University, UT PGE SURI 2020
### Supervised by:
### Michael Pyrcz, Associate Professor, University of Texas at Austin
***
### Introduction
Correlation coefficients and standardization are two elementary concepts from the realm of multivariate statistics (the latter present even in univariate statistics) which seemingly bear no connection to the outcome of complicated algorithms such as those of multidimensional scaling (MDS). Nevertheless, correlation and standardization fundamentally influence how events in a spatial data set behave (e.g. how far apart they are situated - relevant to MDS purposes). This workflow seeks to qualitatively demonstrate through an user-controlled interactive how combinations of correlation coefficients and standardization do impact the visualization and accuracy of MDS projections.

***
### Theory
**Correlation coefficients**, also known by their full name of Pearson's Product Moment Correlation Coefficient, are bivariate measures of the degree of linear dependency between two features. They are defined as:

<br>
\begin{equation}
\rho_{xy} = \frac{\sum_{i=1}^{n}{(x_{i}-\bar{x})}{(y_{i}-\bar{y})}}{(n-1)\sigma_{x}\sigma_{y}}
\end{equation}

where $\rho_{xy}$ is the correlation coefficient between features $x$ and $y$, $n$ is the total number of events, $\bar{x}$ and $\bar{y}$ are the means for features $x$ and $y$, and $\sigma_{x}$ and $\sigma_{y}$ are the standard deviations of features $x$ and $y$. 

Note that $\rho$ can vary in the range $[-1,1]$, where $1$ illustrates perfect positive correlation (linear behavior - along $y=x$) and $-1$ shows perfect negative correlation (negative linear behavior - along $y=-x$).

Rather than a measure, **standardization** provides a univariate process by which different features are transferred to a common range while retaining the significance of their original values. To standardize a data point, apply the following formula:

<br>
\begin{equation}
x_{i,standardized} = \frac{x_{i} - \bar{x}}{\sigma_{x}}
\end{equation}

where $x_{i}$ is the value of feature $x$ for event $i$, $\bar{x}$ is the mean of feature $x$, and $\sigma_{x}$ is the standard deviation of feature $x$. 

Note that standardization is especially useful when dealing with features of different orders of magnitude because it creates Gaussian distributions of all variables (mean of $0$ and variance of $1$). This prevents one feature from outweighing another in processes like multidimensional scaling.

**Multidimensional scaling (MDS)** is a powerful application from inferential machine learning employed to project high-dimensional data sets onto lower dimensions. The resulting $2D$ or $3D$ projection may reveal clusters, trends or event outliers which help describe the data set and determine events worth further study. A key concern for MDS is to preserve original pairwise distances in the resulting projection (i.e. decreasing distortions). Several types of MDS exist, such as metric and nonmetric MDS, each with their own algorithms to reduce projection stress. There is much more to learn on MDS, so we encourage the reader to check out the "MDS_cities_workflow" demonstration which can be found under the References section for a more detailed, step-by-step walkthrough of MDS. In this interactive, we will utilize the SMACOF algorithm for nonmetric MDS provided by the *sklearn* Python package.

***
### Objective
In this demonstration, we will determine if different correlation coefficients and standardization influence the outcome and the accuracy (i.e. how it affects a projection's stress) in multidimensional scaling projections. To do so, we will need to generate realistic spatial data sets and conduct multidimensional scaling on them.
***
### Structure of workflow
This workflow was designed to *allow the user full control and freedom to try out different configurations of correlation coefficients and standardization* in an effort to identify the impact on MDS. As such, this interactive workflow includes not only the code used to generate and analyze spatial data, but also widgets which provide inputs to the processing functions. The interactive utilizes a *combination of the Python packages Ipywidgets and Matplotlib*.

First, we will walk through the functions which generate sample data and project into MDS space. Then, we present the widget configurations. Next, we invite the user to experiment with different widget combinations and identify potential impacts on the MDS projections. Finally, we list a set of conclusions observed from multiple runs of the interactive. *Note that the code for data set generation is computationally expensive and so the interactive might not produce an immediate answer depending on your machine's processing capability.*

#### Import libraries
We begin by importing some standard Python libraries. Note that typically these would be called within the functions below, but because the interactive supposes repeated use of these libraries, we import these packages outside the functions for greater efficiency. Most of these libraries should come installed if you have Anaconda or other environment software.

In [1]:
import numpy as np
import pandas as pd
import geostatspy.geostats as geostats    # reimplimentation of GSLIB algorithms in Python
import geostatspy.GSLIB as GSLIB          # GSLIB visualization, subroutines and algorithm wrappers
from copy import copy                     # deep copies
import sys, os

from sklearn.manifold import MDS          # Multidimensional scaling
import matplotlib.pyplot as plt           # Plotting and settings
from mpl_toolkits.mplot3d import Axes3D   # 3D scatter plots

# Interactive resources:
from ipywidgets import interactive, interact_manual
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox

If you get a package import error, you may have to first install some of these packages. This can usually be accomplished by opening up a command window on Windows and then typing 'python -m pip install [package-name]'. More assistance is available with the respective package docs.

#### *make_sample_data()* Function
The function below creates the spatial data sets which undergo multidimensional scaling. Note that the code below inherits from an interactive workflow written by Dr. Michael Pyrcz (see more detail on function docstring). *make_sample_data* will create a 4 feature (facies, porosity, permeability, and acoustic impedance) data set, where porosity is the truth model with a mean of $0$ and variance of $1$, *corr1* is the correlation coefficient between porosity and permeability, and *corr2* is the correlation coefficient between porosity and acoustic impedance. The spatial data set is generated by a combination of regular, random, and rejection sampling methods which aim to emulate a real-world sample as much as possible.

In [2]:
def make_sample_data(corr1, corr2, standardize=True):
    '''
    Function used to create nonlinear, multivariate spatial data sets through a combination of
    regular, random, and biased sampling techniques. Output data set contains sample Cartesian coordinates,
    porosity, permeability, acoustic impedance, and facies information.
    
    :param corr1: correlation coefficient between permeability and porosity (truth model)
    :type: float in range [-1,1] with increments of 0.05
    :param corr2: correlation coefficient beween acoustic impedance and porosity (truth model)
    :type: float in range [-1,1] with increments of 0.05
    :param standardize: default True to standardize features of data set
    :type: boolean
    :return:biased_samples: 4-feature data set generated by regular, random, and biased sampling methods
           :type: Pandas DataFrame
    
    Note: All credit to Dr. Michael Pyrcz, Associate Professor at UT Austin; the code below is modified from his 
    "make_nonlinear_MV_spatial_data" workflow available on his PythonNumericalDemos GitHub repository at GeoStatsGuy 
    (https://github.com/GeostatsGuy/PythonNumericalDemos).
    
    '''
    
    #***********************************************************************************************************************
    # BLOCK / ENABLE PRINT FUNCTIONS
    #***********************************************************************************************************************
    
    # To prevent unnecessary outputs from geostats.sgsim() calls
    
    # Disable printing
    def blockPrint():
        sys.stdout = open(os.devnull, 'w')

    # Restore printing
    def enablePrint():
        sys.stdout = sys.__stdout__

    #***********************************************************************************************************************
    # TRUTH MODEL PARAMETERS
    #***********************************************************************************************************************

    # Truth Model Parameters 
    seed = 73073                              # random number seed for repeatable random samples and rejection sampler 
    name = 'Por'                              # name of the feature of interest
    nug = 0.0                                 # nugget effect - variogram model
    it = 1                                    # type of structure (1 -spherical, 2 - exponential, 3 - Gaussian)
    azi = 45.0                                # primary direction/azimuth of continuity (0 = y positive, 90 = x positive)
    hmaj = 800                                # variogram range in the major direction
    hmin = 300                                # variogram range in the minor direction (major direction + 90)

    # Make the variogram object
    vario = GSLIB.make_variogram(nug,nst=1,it1=it,cc1=1.0,azi1=azi,hmaj1=hmaj,hmin1=hmin) # make model object

    # Make the dummy dataset (should be outside the range of correlation of the model)
    df = pd.DataFrame({'X':np.full(100,-9999),'Y':np.full(100,-9999),name:np.random.normal(0.0,1.0,100)})

    # Truth Model Grid Parameters
    nx = 100; ny = 100                        # number of cells in the x and y directions
    xsiz = 10.0; ysiz = 10.0                  # size of the cells in the x and y directions
    xmn = xsiz * 0.5; ymn = ysiz * 0.5        # assume origin at 0,0, calculate the 1 cell centroid
    xmin = xmn - 0.5*xsiz; ymin = ymn - 0.5*ysiz # assume origin at 0,0, calculate the min and max x/y coordinates
    xmax = xmin + nx * xsiz; ymax = ymin + ny * ysiz;
    vmin = -3; vmax = 3                   # feature min and max for color bars
    sim_ns = np.zeros([6,ny,nx])

    #***********************************************************************************************************************
    # GAUSSIAN SPATIAL REALIZATIONS
    #***********************************************************************************************************************
    
    blockPrint()
    
    # Calculate the Simulated Truth Model over the Specified Grid and Variogram 
    sim_ns[0,:,:] = geostats.sgsim(df,'X','Y',name,wcol=-1,scol=-1,tmin=-9999,tmax=9999,itrans=0,ismooth=0,dftrans=0,tcol=0,
                twtcol=0,zmin=-3.0,zmax=3.0,ltail=1,ltpar=-3.0,utail=1,utpar=3.0,nsim=1,
                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed=73073,
                ndmin=0,ndmax=100,nodmax=10,mults=0,nmult=2,noct=-1,radius=hmaj,radius1=1,sang1=0,
                mxctx=10,mxcty=10,ktype=0,colocorr=0.0,sec_map=0,vario=vario); # corr = 0 bc Por is the feature of interest
    sim_ns[0,:,:] = GSLIB.affine(sim_ns[0,:,:],0.0,1.0); # correct the mean and variance


    sim_ns[1,:,:] = geostats.sgsim(df,'X','Y',name,wcol=-1,scol=-1,tmin=-9999,tmax=9999,itrans=0,ismooth=0,dftrans=0,tcol=0,
                twtcol=0,zmin=-3.0,zmax=3.0,ltail=1,ltpar=-3.0,utail=1,utpar=3.0,nsim=1,
                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed=73074,
                ndmin=0,ndmax=100,nodmax=10,mults=0,nmult=2,noct=-1,radius=hmaj,radius1=1,sang1=0,
                mxctx=10,mxcty=10,ktype=4,colocorr=corr1,sec_map=sim_ns[0,:,:],vario=vario); # corr to Por
    sim_ns[1,:,:] = GSLIB.affine(sim_ns[1,:,:],0.0,1.0); # correct the mean and variance

    sim_ns[2,:,:] = geostats.sgsim(df,'X','Y',name,wcol=-1,scol=-1,tmin=-9999,tmax=9999,itrans=0,ismooth=0,dftrans=0,tcol=0,
                twtcol=0,zmin=-3.0,zmax=3.0,ltail=1,ltpar=-3.0,utail=1,utpar=3.0,nsim=1,
                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed=73075,
                ndmin=0,ndmax=100,nodmax=10,mults=0,nmult=2,noct=-1,radius=hmaj,radius1=1,sang1=0,
                mxctx=10,mxcty=10,ktype=4,colocorr=corr2,sec_map=sim_ns[0,:,:],vario=vario); # corr to Por
    sim_ns[2,:,:] = GSLIB.affine(sim_ns[2,:,:],0.0,1.0); # correct the mean and variance

    sim_ns[3,:,:] = geostats.sgsim(df,'X','Y',name,wcol=-1,scol=-1,tmin=-9999,tmax=9999,itrans=0,ismooth=0,dftrans=0,tcol=0,
                twtcol=0,zmin=-3.0,zmax=3.0,ltail=1,ltpar=-3.0,utail=1,utpar=3.0,nsim=1,
                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed=73076,
                ndmin=0,ndmax=100,nodmax=10,mults=0,nmult=2,noct=-1,radius=hmaj,radius1=1,sang1=0,
                mxctx=10,mxcty=10,ktype=0,colocorr=0.0,sec_map=0,vario=vario); # corr = 0 bc Por is the feature of interest
    sim_ns[3,:,:] = GSLIB.affine(sim_ns[3,:,:],0.0,1.0); # correct the mean and variance

    sim_ns[4,:,:] = geostats.sgsim(df,'X','Y',name,wcol=-1,scol=-1,tmin=-9999,tmax=9999,itrans=0,ismooth=0,dftrans=0,tcol=0,
                twtcol=0,zmin=-3.0,zmax=3.0,ltail=1,ltpar=-3.0,utail=1,utpar=3.0,nsim=1,
                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed=73077,
                ndmin=0,ndmax=100,nodmax=10,mults=0,nmult=2,noct=-1,radius=hmaj,radius1=1,sang1=0,
                mxctx=10,mxcty=10,ktype=4,colocorr=corr1,sec_map=sim_ns[3,:,:],vario=vario); # corr to Por
    sim_ns[4,:,:] = GSLIB.affine(sim_ns[4,:,:],0.0,1.0); # correct the mean and variance

    sim_ns[5,:,:] = geostats.sgsim(df,'X','Y',name,wcol=-1,scol=-1,tmin=-9999,tmax=9999,itrans=0,ismooth=0,dftrans=0,tcol=0,
                twtcol=0,zmin=-3.0,zmax=3.0,ltail=1,ltpar=-3.0,utail=1,utpar=3.0,nsim=1,
                nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed=73078,
                ndmin=0,ndmax=100,nodmax=10,mults=0,nmult=2,noct=-1,radius=hmaj,radius1=1,sang1=0,
                mxctx=10,mxcty=10,ktype=4,colocorr=corr2,sec_map=sim_ns[3,:,:],vario=vario); # corr to Por
    sim_ns[5,:,:] = GSLIB.affine(sim_ns[5,:,:],0.0,1.0); # correct the mean and variance

    
    enablePrint()
    
    #***********************************************************************************************************************
    # GAUSSIAN TO PROPERTY DISTRIBUTION (SAND)
    #***********************************************************************************************************************

    df_ns = pd.DataFrame({'NS1':sim_ns[0,:,:].flatten(),'NS2':sim_ns[1,:,:].flatten(),
                       'NS3':sim_ns[2,:,:].flatten(),'NS4':sim_ns[3,:,:].flatten(),
                       'NS5':sim_ns[4,:,:].flatten(),'NS6':sim_ns[5,:,:].flatten()})

    df_ns['Por1'] = GSLIB.affine(df_ns['NS1'],13.0,4.5)

    a = 5.5; b = 35; c = 100
    # impose nonlinearity through a quadratic transform of the permeability 
    df_ns['Perm1'] = a * df_ns['NS2'] * df_ns['NS2'] + b * df_ns['NS2'] + c 
    df_ns['Perm1'] = GSLIB.affine(df_ns['Perm1'],4.3,1.0)*100

    a = 5.5; b = 50; c = 100
    df_ns['AI1'] = a * df_ns['NS3'] * df_ns['NS3'] + b * df_ns['NS3'] + c 
    df_ns['AI1'] = GSLIB.affine(df_ns['AI1'],4.0,0.3)*1000

    df_ns['facies1'] = np.full(len(df_ns),1)

    #***********************************************************************************************************************
    # GAUSSIAN TO PROPERTY DISTRIBUTION (SHALE)
    #***********************************************************************************************************************

    df_ns['Por2'] = GSLIB.affine(df_ns['NS4'],7.0,2.5)

    a = 5.5; b = 35; c = 100
    # impose nonlinearity through a quadratic transform of the permeability 
    df_ns['Perm2'] = a * df_ns['NS5'] * df_ns['NS5'] + b * df_ns['NS5'] + c 
    df_ns['Perm2'] = GSLIB.affine(df_ns['Perm2'],2.0,0.3)*100

    a = 5.5; b = 50; c = 100
    df_ns['AI2'] = a * df_ns['NS6'] * df_ns['NS6'] + b * df_ns['NS6'] + c 
    df_ns['AI2'] = GSLIB.affine(df_ns['AI2'],5.0,0.4)*1000

    df_ns['facies2'] = np.full(len(df_ns),0)


    #***********************************************************************************************************************
    # MAKE FACIES MODEL
    #***********************************************************************************************************************

    # Sequential Indicator Simulation with Simple Kriging Multiple Realizations 
    nx = 100; ny = 100; xsiz = 10.0; ysiz = 10.0; xmn = 5.0; ymn = 5.0; nxdis = 1; nydis = 1
    ndmin = 0; ndmax = 10; nodmax = 10; radius = 400; skmean = 0
    tmin = -999; tmax = 999
    df_dummy_facies = pd.DataFrame({'X':np.full(100,-9999),'Y':np.full(100,-9999),'Facies':np.full(100,0)})
    dummy_trend = np.zeros((10,10,2))            # the current version requires trend input - if wrong size it is ignored 

    ncut = 2                                   # number of facies
    thresh = [0,1]                             # the facies categories (use consisten order)
    gcdf = [0.4,0.6]                           # the global proportions of the categories
    varios = []                                # the variogram list
    varios.append(GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=45,hmaj1=300,hmin1=100)) # shale indicator variogram
    varios.append(GSLIB.make_variogram(nug=0.0,nst=1,it1=1,cc1=1.0,azi1=45,hmaj1=300,hmin1=100)) # sand indicator variogram

    sisim = geostats.sisim(df_dummy_facies,'X','Y','Facies',ivtype=0,koption=0,ncut=2,thresh=thresh,gcdf=gcdf,trend=dummy_trend,
                   tmin=tmin,tmax=tmax,zmin=0.0,zmax=1.0,ltail=1,ltpar=1,middle=1,mpar=0,utail=1,utpar=2,
                   nx=nx,xmn=xmn,xsiz=xsiz,ny=ny,ymn=ymn,ysiz=ysiz,seed = 73073,
                   ndmin=ndmin,ndmax=ndmax,nodmax=nodmax,mults=1,nmult=3,noct=-1,radius=radius,ktype=0,vario=varios)

    #***********************************************************************************************************************
    # MERGE CONTINUOUS PROPERTIES BY FACIES (COOKIE CUTTER)
    #***********************************************************************************************************************

    por1 = df_ns['Por1'].values.reshape([ny,nx])
    perm1 = df_ns['Perm1'].values.reshape([ny,nx])
    AI1 = df_ns['AI1'].values.reshape([ny,nx])

    por2 = df_ns['Por2'].values.reshape([ny,nx])
    perm2 = df_ns['Perm2'].values.reshape([ny,nx])
    AI2 = df_ns['AI2'].values.reshape([ny,nx])

    facies = np.zeros([ny,nx])
    por = np.zeros([ny,nx])
    perm = np.zeros([ny,nx])
    AI = np.zeros([ny,nx])

    for iy in range(0,ny):
        for ix in range(0,nx):
            if sisim[iy,ix] > 0.5:   # current location is assumed to be sand
                facies[iy,ix] = 1
                por[iy,ix] = por1[iy,ix];
                perm[iy,ix] = perm1[iy,ix];
                AI[iy,ix] = AI1[iy,ix]
            else:                      # current location is assumed to be shale
                facies[iy,ix] = 0
                por[iy,ix] = por2[iy,ix];
                perm[iy,ix] = perm2[iy,ix];
                AI[iy,ix] = AI2[iy,ix]

    #***********************************************************************************************************************
    # REGULAR, RANDOM, REJECT SAMPLE FUNCTIONS
    #***********************************************************************************************************************

    def regular_sample_MV(array,array2,array3,array4,xmin,xmax,ymin,ymax,spacing,
                          minsx=xmin,maxsx=xmax,minsy=ymin,maxsy=ymax,name='Value',name2='Value2',name3='Value3',name4="Value4"):
        x = []; y = []; v = []; v2 = []; v3 = []; v4 = []
        nx = array.shape[1]; ny = array.shape[0]
        xsiz = (xmax-xmin)/nx; ysiz = (ymax-ymin)/ny
        xmn = xmin + 0.5*xsiz; ymn = ymin + 0.5*ysiz
        xx, yy = np.meshgrid(np.arange(xmin, xmax, spacing), np.arange(ymax, ymin, -1 * spacing))
        xx = xx + spacing*0.5; yy = yy - spacing*0.5
        for ix,iy in np.ndindex(xx.shape):
            iix = geostats.getindex(nx,xmn,xsiz,xx[iy,ix])
            iiy = geostats.getindex(ny,ymn,ysiz,yy[iy,ix])
            if yy[iy,ix] >= minsy and yy[iy,ix] <= maxsy: 
                if xx[iy,ix] >= minsx and xx[iy,ix] >= minsx: 
                    x.append(xx[iy, ix])
                    y.append(yy[iy, ix])
                    v.append(array[ny - iiy - 1, iix])
                    v2.append(array2[ny - iiy - 1, iix])
                    v3.append(array3[ny - iiy - 1, iix])
                    v4.append(array4[ny - iiy - 1, iix])
        df = pd.DataFrame(np.c_[x, y, v, v2, v3, v4], columns=["X", "Y", name,name2,name3,name4])
        return df

    def random_sample_MV(array,array2,array3,array4,xmin,xmax,ymin,ymax,nsamp=10,minsx=xmin,maxsx=xmax,minsy=ymin,maxsy=ymax,name='Value',name2='Value2',name3='Value3',name4='Value4'):
        x = []; y = []; v = []; v2 = []; v3 = []; v4 = []
        ny, nx = array.shape   
        xx, yy = np.meshgrid(
            np.arange(xmin, xmax,(xmax-xmin)/float(nx)), np.arange(ymax - 1, ymin - 1, -1 * (ymax-ymin)/float(ny))
        )
        mask = np.zeros([ny,nx])
        for iy in range(0,ny):
            for ix in range(0,nx):
                if xx[iy,ix] >= minsx and xx[iy,ix] <= maxsx:
                    if yy[iy,ix] >= minsy and yy[iy,ix] <= maxsy:
                        mask[iy,ix] = 1.0

        if nsamp*1.2 > np.sum(mask):
            print('ERROR - too few locations available for number of samples requested!') 
            return pd.DataFrame()
        isamp = 0
        while isamp < nsamp:
            sample_index = np.random.choice(range(nx * ny), 1)
            iy = int(sample_index[0] / ny)
            ix = sample_index[0] - iy * nx
            if mask[iy,ix] == 1:    
                if xx[iy,ix] >= xmin and xx[iy,ix] <= xmax:
                    if yy[iy,ix] >= ymin and yy[iy,ix] <= ymax:        
                        x.append(xx[iy, ix])
                        y.append(yy[iy, ix])
                        v.append(array[iy, ix])
                        v2.append(array2[iy, ix])
                        v3.append(array3[iy, ix]) 
                        v4.append(array4[iy, ix])      
                        mask[iy,ix] = 0.0
                        isamp = isamp + 1
        df = pd.DataFrame(np.c_[x, y, v, v2, v3, v4], columns=["X", "Y", name,name2,name3,name4])
        return df

    def rejection_sample(df,vcol,frac,wt_min,wt_max):
        value = [np.min(df[vcol].values),np.max(df[vcol].values)]
        wt = [wt_min,wt_max]
        df_copy = df.copy(deep = True)
        df_copy['weights'] = np.interp(df[vcol],value,wt)
        df_sample = df_copy.sample(frac = frac, replace = False, weights = 'weights')
        return df_sample

    #***********************************************************************************************************************
    # SAMPLE FROM TRUTH MODEL FOR EACH FEATURE (USE OF REGULAR, RANDOM, REJECT SAMPLE FUNCTIONS)
    #***********************************************************************************************************************

    # Data Set Sampling Paramters
    spacing = 30                              # spacing of regular samples
    nrandom_sample = 400                      # number of random samples
    wt_min = 0.1; wt_max = 1.0                # weigths of minimum and maximum feature values - control on degree of bias 
    bias_frac = 0.7                           # fraction of dataset to remove with rejection sampler
    seed = 73073                              # random number seed
    vmin = 0.0; vmax = 25.0

    np.random.seed(seed = seed)               # set the random number seed for repeatability of samples

    # Regular Sampling
    regular_samples = regular_sample_MV(por,perm,AI,facies,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,spacing=spacing,
                             minsx=xmin,maxsx=xmax,minsy=ymin,maxsy=ymax,name="Por",name2="Perm",name3="AI",name4="Facies")

    # Random Sampling
    random_samples = random_sample_MV(por,perm,AI,facies,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,
                                   nsamp = nrandom_sample,minsx=xmin,maxsx=xmax,minsy=ymin,maxsy=ymax,
                                   name="Por",name2="Perm",name3="AI",name4="Facies")

    # Concatenate (Combine) Regular and Random Samples into One DataFrame  
    samples = pd.concat([regular_samples,random_samples])

    # Rejection Sampler
    biased_samples = rejection_sample(samples,"Por",1-bias_frac,wt_min,wt_max)

    # Remove Dupicates 
    biased_samples = biased_samples[np.invert(biased_samples.duplicated(subset=['X','Y'],keep='first'))] 

    # Drop the weights column
    biased_samples = biased_samples.drop('weights',axis=1)   
    
    #***********************************************************************************************************************
    # STANDARDIZATION OF RESULT SAMPLE SET
    #***********************************************************************************************************************
    
    if standardize == True:
        biased_samples = (biased_samples - biased_samples.mean())/biased_samples.std()
        return biased_samples
    elif standardize == False:
        return biased_samples
    else:
        raise TypeError('"standardize" parameter must be boolean')

#### *make_MDS()* Function
Now that we have an utility which creates a spatial data set from user-selected correlation coefficients, we can code up another function to do MDS on the data set and produce plots of the MDS projection and its accuracy. Observe that most of the function below is dedicated to calculating pairwise distances and the Kruskal stress for the second plot. The core of the (non-metric) multidimensional scaling analysis is executed with the *sklearn.manifold.MDS* module.

In [3]:
def make_MDS(sample_df, n_dim, corr1, corr2, standardize=True):
    '''
    Function that computes non-metric MDS of selected sample features and outputs plots of MDS projection and
    model accuracy check with Kruskal stress. Relies on sklearn.manifold.MDS.
    
    :param sample_df: spatial data set with select features for MDS analysis
    :type: Pandas DataFrame
    :param n_dim: number of dimensions of MDS projection
    :type: str '2D' or '3D'
    :param corr1: correlation coefficient between porosity and permeability (for plot label purposes)
    :type: float in range [-1,1] with increments of 0.05
    :param corr2: correlation coefficient between porosity and acoustic impedance (for plot label purposes)
    :type: float in range [-1,1] with increments of 0.05
    :param standardize: default True to standardize data set by feature (for plot label purposes)
    :type: boolean
    :return:scatter plot of MDS projection
           :type: 2D or 3D scatter plot
           :plot of original versus projected pairwise distances w/ stress
           :type: 2D scatter plot
    
    '''
    
    sample = sample_df.to_numpy()                                               # Convert to NumPy array
    
    #***********************************************************************************************************************
    # NONMETRIC MDS
    #***********************************************************************************************************************
    
    embedding = MDS(n_components=int(n_dim.replace('D','')))
    proj_coords = embedding.fit_transform(sample)                               # Compute and store the MDS coordinates
    
    #***********************************************************************************************************************
    # CALCULATE ORIGINAL DISTANCES
    #***********************************************************************************************************************
    
    orig_dists = np.zeros(shape=(np.shape(proj_coords)[0],np.shape(proj_coords)[0]))
    
    for i in range(np.shape(sample)[0]):                                        # Span rows
        for j in range(np.shape(sample)[0]):                                    # Span columns
            if i==j:                                                            # Distance of event to itself is null
                continue
            else:
                inside = 0
                for k in range(np.shape(sample)[1]):                            # Span features
                    inside += (sample[i,k] - sample[j,k])**2
                
                orig_dists[i,j] = np.sqrt(inside)
    
    #***********************************************************************************************************************
    # CALCULATE PROJECTED DISTANCES
    #***********************************************************************************************************************
    
    proj_dists = np.zeros(shape=(np.shape(proj_coords)[0],np.shape(proj_coords)[0]))    # MDS projected distances

    for i in range(np.shape(proj_coords)[0]):                                           # Span rows
        for j in range(np.shape(proj_coords)[0]):                                       # Span columns
            if i==j:
                continue                                                                # Distance of event to itself is null
            else:
                inside = 0
                for k in range(np.shape(proj_coords)[1]):                               # Span features
                    inside += (proj_coords[i,k] - proj_coords[j,k])**2  
                
                proj_dists[i,j] = np.sqrt(inside)
       
    #***********************************************************************************************************************
    # STRESS
    #***********************************************************************************************************************
    num = 0; den = 0                                                            # Preallocating numerator and denominator

    for i in range(np.shape(proj_coords)[0]):                                   # Span rows
        for j in range(np.shape(proj_coords)[0]):                               # Span columns
            if i==j:                                                            # num, den would be 0 so skip for efficiency
                continue
            else:
                num += (proj_dists[i,j] - orig_dists[i,j])**2
                den += (orig_dists[i,j])**2
            
    stress = np.sqrt(num/den)

    #***********************************************************************************************************************
    # PLOTTING
    #***********************************************************************************************************************
    plt.style.use('ggplot')
    
    plt.figure(figsize=(14,10))    # MDS projection
    
    if np.shape(proj_coords)[1]==2:                                                                       # 2D projection
        plt.scatter(proj_coords[:,0],proj_coords[:,1],marker='o',s=10, color='blue');
    else:                                                                                                 # 3D projection
        ax = plt.axes(projection='3d')
        ax.scatter3D(proj_coords[:,0],proj_coords[:,1],proj_coords[:,2],marker='o',s=10, color='blue');
        ax.set_zlabel('Third MDS Dimension',size=20);
    
    plt.xlabel('First MDS Dimension',size=20); plt.ylabel('Second MDS Dimension',size=20);
    plt.suptitle('{dim}D MDS Projection of Subsurface Sample'.format(dim=int(n_dim.replace('D',''))),size=25)
    plt.title('Correlation Coefficients: {c1},{c2} \n Standardization: {s}'.format(c1=corr1,c2=corr2,s=standardize),size=15)
    
    plt.figure(figsize=(14,10))    # Original vs projected pairwise distances and Kruskal stress
    
    plt.scatter(orig_dists,proj_dists,label='Projected vs Original Distances', color='red',s=3);
    plt.plot(np.max(orig_dists)*np.array([0,1]),np.max(orig_dists)*np.array([0,1]),color='black',linewidth=2,label='Null Stress (y=x)');
    plt.title('Projected MDS versus Original Distances w/Stress',size=25)
    plt.xlabel('Original Distances',size=20); plt.ylabel('Projected Distances',size=20);   
    plt.text(0.1*np.max(orig_dists),0.9*np.max(orig_dists), 'Kruskal Stress = \n %.3f' %stress, size = 15, horizontalalignment = 'center', \
         bbox=dict(facecolor='none', edgecolor='black'))
    plt.legend(loc='lower right');

#### Widgets and *widget2MDS()* Function
The last step before launching the interactive is to code up the widgets which will provide inputs to the functions above, and to create a function to encapsulate *make_sample_data()* and *make_MDS()*. This global function which we call *widget2MDS()* does not perform any additional processing of the sample data or the MDS projection. Rather, it just helps call the interactive.

In [4]:
header = widgets.Text(value='Impact of Correlation and Standardization in Multidimensional Scaling',\
                      layout=Layout(width='900px', height='50px',justify_content='center'))

style = {'description_width': 'initial'}

corr1 = widgets.FloatSlider(min = -1.0, max = 1.0, value = 0, step = 0.05, description = 'Permeability Correlation Coefficient',\
                            orientation='horizontal',continuous_update=False,style=style,layout=Layout(width='500px', height='50px'))
corr1.style.handle_color = 'red'

corr2 = widgets.FloatSlider(min = -1.0, max = 1.0, value = 0, step = 0.05, description = 'Acoustic Imp. Correlation Coefficient',\
                            style=style,orientation='horizontal',continuous_update=False,layout=Layout(width='500px', height='50px'))
corr2.style.handle_color = 'blue'

standardize = widgets.Checkbox(value=False,description='Standardize by feature?',disabled=False,indent=False,layout=Layout(width='400px', height='50px'))

n_dim = widgets.Dropdown(options=['2D', '3D'],value='2D',description='Number of dimensions in MDS projection:',disabled=False,\
                         style=style,layout=Layout(width='400px', height='50px'))


def widget2MDS(corr1, corr2, n_dim, standardize=True):
    '''
    Function to encapsulate make_sample_data() and make_MDS(). 
    Takes widget inputs and produces MDS projections and Kruskal stress plot.
    
    :param corr1: correlation coefficient between permeability and porosity (truth model)
    :type: float in range [-1,1] with increments of 0.05
    :param corr2: correlation coefficient beween acoustic impedance and porosity (truth model)
    :type: float in range [-1,1] with increments of 0.05
    :param standardize: default True to standardize features of data set
    :type: boolean
    :param n_dim: number of dimensions of MDS projection
    :type: str '2D' or '3D'
    :return:scatter plot of MDS projection
           :type: 2D or 3D scatter plot
           :plot of original versus projected pairwise distances w/ stress
           :type: 2D scatter plot
    '''
    
    sample_df = make_sample_data(corr1=corr1, corr2=corr2, standardize=standardize)
    
    make_MDS(sample_df=sample_df, n_dim=n_dim, corr1=corr1, corr2=corr2, standardize=standardize)
    

# Links widget inputs to widget2MDS()
play_button = interactive(widget2MDS,{'manual':True},corr1=corr1,corr2=corr2,n_dim=n_dim,standardize=standardize)

***
### Interactive 
With all functions and widgets properly set up and linked together, it is time to run our interactive. Note that much more could have been done to improve the visualization of the widgets, and we invite the user willing to collaborate to do so.

##### Instructions:
 - Red dial: refers to the **correlation coefficient between the sample's permeability and porosity** and ranges from -1 (for perfect inverse behavior) to 1 (for perfect direct proportionality) with increments of 0.05;
 - Blue dial: refers to the **correlation coefficient between acoustic impedance and porosity**; 
 - Number of dimensions in MDS projection: in **how many dimensions to display the MDS projection** scatter plot (2D or 3D);
 - Standardize by Feature: when checked **will standardize the selected features**;

Once you have your settings picked, make sure to press **"Run Interact"** otherwise the interactive will not start. Enjoy!

In [5]:
display(header, play_button)

Text(value='Impact of Correlation and Standardization in Multidimensional Scaling', layout=Layout(height='50px…

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Permeability Correlation Co…

***
### Conclusion
By running the interactive above for different combinations of initial settings (large, medium, and small correlation coefficients; standardized and non-standardized data), the user can draw three main qualitative conclusions:


1) Standardizing data by feature facilitates the appearance of clusters in spite of the correlation coefficients and projection dimensionality chosen;

2) For both standardized and non-standardized data sets, the variance in projection stress increases with the magnitude of the correlation coefficients. For example, 3 iterations with standardized data and large correlation coefficients ($|\rho|=1$) yield stress values of $0.179$, $0.190$, and $0.145$, while 3 iterations with standardized data and small correlation coefficients ($|\rho|=0.1$) yield stress values of $0.191$, $0.194$, and $0.192$;

3) The projection stress decreases for greater projection dimensionality. Hence, a $3D$ MDS projection will certainly distort less the original pairwise distances than a $2D$ MDS projection.

***
### References
The sources below provide further reading and video material on correlation coefficients, standardization, and MDS:

1. [MDS_cities_workflow](https://github.com/AlanScherman/SURI2020/blob/master/MDS%20-%20Cities%20demo/MDS_cities_workflow.ipynb)
2. [Interpreting MDS projections](http://www.analytictech.com/borgatti/mds.htm)
3. [sklearn.manifold.MDS Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html) 
4. [Dr. Michael Pyrcz's lecture on correlation](https://www.youtube.com/watch?v=wZwYEDqB4A4&list=PLG19vXLQHvSB-D4XKYieEku9GQMQyAzjJ&index=20)
5. [Dr. Michael Pyrcz's "make_nonlinear_MV_spatial_data" workflow](https://github.com/GeostatsGuy/PythonNumericalDemos/blob/master/make_nonlinear_MV_spatial_data.ipynb)